In [4]:
import pandas as pd
import numpy as np
from vega_datasets import data
import altair as alt
import streamlit as st

In [5]:
df = pd.read_csv('Senate_dataset/1976-2020-senate.csv', encoding = "ISO-8859-1",index_col="state_fips")
df["state_fips"] = df.index

In [6]:
# drop unused columns and nulls
df = df.drop(columns=['mode', 'version', 'unofficial','writein','special','stage'])
df = df.dropna()
df.shape

(3030, 13)

In [7]:
# drop counties reporting zero total votes
df = df.loc[df['totalvotes'] != 0]
df.shape

(3030, 13)

In [8]:
# get % of votes each party received in each county
temp = []
for index, row in df.iterrows():
    temp.append(round(row['candidatevotes'] / row['totalvotes'], 4))
df['percentage'] = temp

In [30]:
df

,index,year,state,state_po,state_fips,state_cen,state_ic,office,district,candidate,party_detailed,candidatevotes,totalvotes,party_simplified,percentage
0,0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,SAM STEIGER,REPUBLICAN,321236,741210,REPUBLICAN,0.4334
1,1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,WM. MATHEWS FEIGHAN,INDEPENDENT,1565,741210,OTHER,0.0021
2,2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,DENNIS DECONCINI,DEMOCRAT,400334,741210,DEMOCRAT,0.5401
3,3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,ALLAN NORWITZ,LIBERTARIAN,7310,741210,LIBERTARIAN,0.0099
4,4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,BOB FIELD,INDEPENDENT,10765,741210,OTHER,0.0145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,3619,2020,WEST VIRGINIA,WV,54,55,56,US SENATE,statewide,DAVID MORAN,LIBERTARIAN,21155,778918,LIBERTARIAN,0.0272
3026,3625,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,KELLY LOEFFLER,REPUBLICAN,2194848,4483294,REPUBLICAN,0.4896
3027,3626,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,RAPHAEL WARNOCK,DEMOCRAT,2288446,4483294,DEMOCRAT,0.5104
3028,3627,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,DAVID A. PERDUE,REPUBLICAN,2213979,4483241,REPUBLICAN,0.4938


In [9]:
#create new dataframe with desired data config
curr_state = 'ARIZONA'
new_df = []
temp_dict = {}
for index, row in df.iterrows():
    temp_dict['year'] = row['year']
    temp_dict['state'] = row['state']
    temp_dict['state_fips'] = int(row['state_fips'])
    temp_dict['total_no'] = row['totalvotes']
    curr_party = row['party_detailed'][0].lower()
    
    
    if row['state'] == curr_state:
        if curr_party == 'd':
            temp_dict['dem_no'] = row['candidatevotes']
            temp_dict['dem_pct'] = row['percentage']
        elif curr_party == 'r':
            temp_dict['rep_no'] = row['candidatevotes']
            temp_dict['rep_pct'] = row['percentage']
        elif curr_party == 'l':
            temp_dict['lib_no'] = row['candidatevotes']
            temp_dict['lib_pct'] = row['percentage']
        elif curr_party == 'g':
            temp_dict['grn_no'] = row['candidatevotes']
            temp_dict['grn_pct'] = row['percentage']
        else:
            temp_dict['oth_no'] = row['candidatevotes']
            temp_dict['oth_pct'] = row['percentage']
    else:
        copy = temp_dict.copy()
        new_df.append(copy)
        temp_dict.clear()
        
        temp_dict['year'] = row['year']
        temp_dict['state'] = row['state']
        temp_dict['state_fips'] = int(row['state_fips'])
        temp_dict['total_no'] = row['totalvotes']
        curr_party = row['party_detailed'][0].lower()

        if curr_party == 'd':
            temp_dict['dem_no'] = row['candidatevotes']
            temp_dict['dem_pct'] = row['percentage']
        elif curr_party == 'r':
            temp_dict['rep_no'] = row['candidatevotes']
            temp_dict['rep_pct'] = row['percentage']
        elif curr_party == 'l':
            temp_dict['lib_no'] = row['candidatevotes']
            temp_dict['lib_pct'] = row['percentage']
        elif curr_party == 'g':
            temp_dict['grn_no'] = row['candidatevotes']
            temp_dict['grn_pct'] = row['percentage']
        else:
            temp_dict['oth_no'] = row['candidatevotes']
            temp_dict['oth_pct'] = row['percentage']
    
    curr_state = row['state']
    

new_df = pd.DataFrame(new_df)
new_df = new_df.set_index("state_fips")
new_df["state_fips"] = new_df.index

In [10]:
new_df = new_df.replace(np.nan, 0)
new_df

,year,state,total_no,rep_no,rep_pct,oth_no,oth_pct,dem_no,dem_pct,lib_no,lib_pct,grn_no,grn_pct,state_fips
state_fips,,,,,,,,,,,,,,
6,1976,CALIFORNIA,7470586,321236.0,0.4334,10765.0,0.0145,400334.0,0.5401,7310.0,0.0099,0.0,0.0000,6
9,1976,CONNECTICUT,1361666,3748973.0,0.5018,104383.0,0.0140,3502862.0,0.4689,0.0,0.0000,0.0,0.0000,9
10,1976,DELAWARE,224795,785683.0,0.5770,14407.0,0.0106,561018.0,0.4120,0.0,0.0000,0.0,0.0000,10
12,1976,FLORIDA,2857534,125454.0,0.5581,437.0,0.0019,98042.0,0.4361,0.0,0.0000,0.0,0.0000,12
15,1976,HAWAII,302092,1057886.0,0.3702,0.0,0.0000,1799518.0,0.6297,0.0,0.0000,0.0,0.0000,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,2020,TENNESSEE,2959761,276232.0,0.6574,0.0,0.0000,143987.0,0.3426,0.0,0.0000,0.0,0.0000,47
48,2020,TEXAS,11144040,1840926.0,0.6220,6781.0,0.0023,1040691.0,0.3516,0.0,0.0000,0.0,0.0000,48
51,2020,VIRGINIA,4405087,5962983.0,0.5351,0.0,0.0000,4888764.0,0.4387,209722.0,0.0188,81893.0,0.0073,51
